In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from track import get_infected_img
from track import get_activations_at_h
import sys
from pathlib import Path

sys.path.insert(0, '..') # in order to be able to import from scripts.py
from scripts.client import VisAVisClient
from scripts.make_protocol import make_protocol

In [ ]:
PARAMETERS_DEFAULT = {
  "c_rate": 1,
  "e_incr": 1,
  "i_incr": 1,
  "r_incr": 0.0667
}

In [ ]:
folder = "results/pulses"
Path(folder).mkdir(parents=True, exist_ok=True)
intervals = [50, 100, 150, 200, 250, 300]
signal_count = 200
sim_num = 5

In [ ]:
for channel_height in [7]:

    client = VisAVisClient(
        visavis_bin=f'../target/bins/vis-a-vis-{channel_height}',
    )

    for interval in intervals:

        protocol_file_path = make_protocol(
            pulse_intervals = (signal_count) * [interval] + [800],
            duration=4,
            out_folder='./'
        )
        for i in range(sim_num):
            result = client.run(
                parameters_json=PARAMETERS_DEFAULT,
                protocol_file_path= protocol_file_path,
            )
            img = get_infected_img(result.states)
            activations = get_activations_at_h(img, -1)
            activations.to_csv(f"{folder}/activations-{interval}-{i}.csv")

In [ ]:
import seaborn as sns 

fig=plt.figure(figsize=(10,10))
fig.suptitle('Interval at end of canal\n1000 Pulses sent with fixed interval') # or plt.suptitle('Main title')
for i, interval in enumerate(intervals):
        plt.subplot(3, 2, i + 1)
        activations = pd.concat(
                [pd.read_csv(f"{folder}/activations-{interval}-{i}.csv")['seconds'].diff() for i in range(sim_num)]
                )
        sns.histplot(activations, bins = 75, stat='probability')
        plt.title(f"Interval at start {interval}")
plt.tight_layout()
